In [ ]:
import os
import io
import sys
import uuid
import shutil
import atexit
from dotenv import load_dotenv
from flask import Flask, request, abort, send_from_directory, render_template
from pyngrok import ngrok
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextSendMessage, ImageMessage, ImageSendMessage, TextMessage

import tensorflow as tf
from roboflow import Roboflow
import cv2
import numpy as np

# --- 步驟 1: 載入環境變數 ---
load_dotenv()

ACCESS_TOKEN = os.getenv('LINE_CHANNEL_ACCESS_TOKEN')
CHANNEL_SECRET = os.getenv('LINE_CHANNEL_SECRET')
NGROK_AUTHTOKEN = os.getenv('NGROK_AUTHTOKEN')
ROBOFLOW_API_KEY = os.getenv('ROBOFLOW_API_KEY')
LIFF_ID = os.getenv('LINE_LIFF_ID')

# --- Ngrok 啟動與防呆檢查 ---
if not NGROK_AUTHTOKEN:
    print("❌ 錯誤：找不到 NGROK_AUTHTOKEN！")
    print("請檢查您的 .env 檔案中是否已設定 ngrok 的 Authtoken。")
    sys.exit(1)

try:
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    public_url = ngrok.connect(
        5000,
        request_header={"remove": ["ngrok-skip-browser-warning"]}
    ).public_url
    print("✅ ngrok 啟動成功！")
except Exception as e:
    print(f"❌ ngrok 啟動失敗: {e}")
    sys.exit(1)

# --- 步驟 2: 初始化服務 ---
app = Flask(__name__)

# --- ★★★【已修正 NameError】通用路徑定義方式 ★★★ ---
try:
    # 此方法在 .py 腳本中執行時有效
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # 此方法在 Jupyter Notebook 等互動式環境中執行時有效
    BASE_DIR = os.getcwd()

STATIC_FOLDER = os.path.join(BASE_DIR, 'static')
# --- ★★★ 修正結束 ★★★ ---

if not os.path.exists(STATIC_FOLDER):
    os.makedirs(STATIC_FOLDER)

# 設定 Line Bot API
line_bot_api = LineBotApi(ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)
line_bot_api.set_webhook_endpoint(f"{public_url}/callback")
print(f"公開網址 (Webhook URL): {public_url}/callback")

# 程式結束時的清理函式
@atexit.register
def cleanup():
    print("\n伺服器即將關閉，開始執行清理作業...")
    # 檢查 public_url 是否存在，避免 ngrok 啟動失敗時報錯
    if 'public_url' in globals() and public_url:
        ngrok.disconnect(public_url)
        print("✅ ngrok 通道已成功關閉。")
    if os.path.exists(STATIC_FOLDER):
        shutil.rmtree(STATIC_FOLDER)
        print(f"✅ 已成功刪除 '{STATIC_FOLDER}' 暫存資料夾。")

# --- 步驟 3: 載入 AI 模型 (與之前相同，此處省略以節省篇幅) ---
# ... 您的模型載入程式碼 ...
CLASSIFIER_MODEL_PATH = 'best_potato_model.keras'
detection_model = None
classifier_model = None
try:
    classifier_model = tf.keras.models.load_model(CLASSIFIER_MODEL_PATH)
    print(f"✅ 發芽辨識模型 '{CLASSIFIER_MODEL_PATH}' 載入成功。")
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    project = rf.workspace().project("potato-detection-3et6q")
    detection_model = project.version(11).model
    print("✅ Roboflow 物件偵測模型載入成功！")
except Exception as e:
    print(f"❌ 模型載入失敗: {e}")
IMAGE_SIZE = (224, 224)
def resize_image(image_bytes, max_dimension=640):
    nparr = np.frombuffer(image_bytes, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    h, w, _ = img.shape
    if h > max_dimension or w > max_dimension:
        if h > w:
            new_h, new_w = max_dimension, int(w * (max_dimension / h))
        else:
            new_w, new_h = max_dimension, int(h * (max_dimension / w))
        resized_img = cv2.resize(img, (new_w, new_h))
        is_success, buffer = cv2.imencode(".jpg", resized_img)
        if is_success:
            return buffer.tobytes()
    return image_bytes
def classify_potato(img_bgr):
    try:
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img_rgb, IMAGE_SIZE)
        img_array = np.expand_dims(img_resized, axis=0)
        prediction = classifier_model.predict(img_array, verbose=0)
        return prediction[0][0]
    except Exception as e:
        print(f"分類時發生錯誤: {e}")
        return None


# --- 步驟 4: Flask 路由與 Line Bot 訊息處理 (與之前相同) ---
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@app.route('/static/<path:path>')
def send_static(path):
    return send_from_directory(STATIC_FOLDER, path)

@app.route("/liff")
def liff_page():
    return render_template('detector.html', 
                           roboflow_api_key=ROBOFLOW_API_KEY,
                           liff_id=LIFF_ID)

@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
    # ... 您處理圖片的完整函式 ...
    # (與您最初的版本相同，此處省略以節省篇幅)
    user_id = event.source.user_id
    line_bot_api.push_message(user_id, TextSendMessage(text="收到圖片！分析中，請稍後..."))
    message_content = line_bot_api.get_message_content(event.message.id)
    original_image_bytes = message_content.content
    processed_image_bytes = resize_image(original_image_bytes)
    temp_image_name = f"{uuid.uuid4()}.jpg"
    temp_image_path = os.path.join(STATIC_FOLDER, temp_image_name)
    with open(temp_image_path, 'wb') as fd:
        fd.write(processed_image_bytes)
    if not detection_model or not classifier_model:
        line_bot_api.push_message(user_id, TextSendMessage(text="模型載入失敗，無法分析。"))
        return
    try:
        predictions = detection_model.predict(temp_image_path, confidence=40, overlap=30).json()['predictions']
    except Exception as e:
        line_bot_api.push_message(user_id, TextSendMessage(text=f"物件偵測時發生錯誤：\n{e}"))
        return
    if not predictions:
        line_bot_api.push_message(user_id, TextSendMessage(text="未偵測到馬鈴薯！"))
        return
    original_image = cv2.imdecode(np.frombuffer(processed_image_bytes, np.uint8), cv2.IMREAD_COLOR)
    output_image = original_image.copy()
    results_text = []
    potato_count = 0
    for pred in predictions:
        potato_count += 1
        center_x, center_y = int(pred['x']), int(pred['y'])
        width, height = int(pred['width']), int(pred['height'])
        x1, y1 = int(center_x - width / 2), int(center_y - height / 2)
        x2, y2 = int(center_x + width / 2), int(center_y + height / 2)
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(original_image.shape[1], x2), min(original_image.shape[0], y2)
        cropped_potato = original_image[y1:y2, x1:x2]
        if cropped_potato.size == 0:
            continue
        score = classify_potato(cropped_potato)
        if score is not None:
            if score > 0.5:
                label, color = f"Sprouted: {score:.2f}", (0, 0, 255)
                results_text.append(f"馬鈴薯 {potato_count}: 已發芽 (機率: {score:.2%})")
            else:
                label, color = f"Not Sprouted: {1 - score:.2f}", (0, 255, 0)
                results_text.append(f"馬鈴薯 {potato_count}: 未發芽 (機率: {1 - score:.2%})")
            cv2.rectangle(output_image, (x1, y1), (x2, y2), color, 3)
            cv2.putText(output_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    result_image_name = f"result_{temp_image_name}"
    result_image_path = os.path.join(STATIC_FOLDER, result_image_name)
    cv2.imwrite(result_image_path, output_image)
    result_image_url = f"{public_url}/static/{result_image_name}"
    image_message = ImageSendMessage(original_content_url=result_image_url, preview_image_url=result_image_url)
    line_bot_api.push_message(user_id, image_message)
    summary_text = f"偵測到 {len(predictions)} 個馬鈴薯：\n" + "\n".join(results_text)
    line_bot_api.push_message(user_id, TextSendMessage(text=summary_text))

@handler.add(MessageEvent, message=TextMessage)
def handle_text_message(event):
    reply_text = "您好！請傳送一張馬鈴薯圖片進行分析。"
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_text)
    )

# --- 步驟 5: 啟動伺服器 ---
if __name__ == "__main__":
    app.run(port=5000)

✅ ngrok 啟動成功！


Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
Call to deprecated method set_webhook_endpoint. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).set_webhook_endpoint(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.


公開網址 (Webhook URL): https://beda25e6c2f5.ngrok-free.app/callback
✅ 發芽辨識模型 'best_potato_model.keras' 載入成功。
loading Roboflow workspace...
loading Roboflow project...
✅ Roboflow 物件偵測模型載入成功！
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Sep/2025 07:49:16] "POST /callback HTTP/1.1" 200 -
